### Testing JSON Read

In [2]:
import json
import re
import os
import pickle
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_colwidth", None)

In [3]:
predictions = pd.read_parquet("s3://mag-model-data/V2/iteration_2/test_data/predictions_20220117_lr0006_beta025_gamma28_nH6_nL4_firstD2048_secondD1024_thirdD1024.parquet")

In [4]:
predictions.shape

(62517, 10)

In [5]:
extra_data = pd.read_parquet("s3://mag-model-data/V2/raw_test_data/part-00000-tid-1178175042784182311-0b55b08b-d5db-4777-9e6a-572d293f915d-832-1-c000.snappy.parquet")

In [6]:
extra_data.shape

(62524, 10)

In [7]:
abstract_test = pd.read_parquet("s3://mag-model-data/raw_mag_data/test_api_abstracts/part-00000-tid-3075369008470969684-b53b0953-d369-469a-9e2b-aa32d5c70e90-1-1-c000.snappy.parquet")

In [8]:
abstract_test.shape

(2, 2)

In [9]:
all_test_data = predictions \
.merge(extra_data.drop(['publication_date'], axis=1), how='left', on='paper_id')

In [10]:
data = all_test_data.head(3).merge(abstract_test, how='left', on='paper_id')

In [11]:
data.head()

,paper_id,publication_date,doc_type_tok,journal_tok,target_tok,paper_title_tok,abstract_tok,predictions,scores,target_test,doc_type,original_title,clean_title,journal_name,abstract_length,abstract,clean_abstract,topics,indexed_abstract
0,18863665,2011-01-01,[8],[2],"[16572, 48550, 52758, 32648, 9593, 44014, 42648, 55971, 27479, 1356, 59810]","[239113, 92190, 154991, 132270, 265955, 114561, 51847, 63104, 256479]","[154201, 47075, 139417, 222311, 200157, 90902, 11995, 148679, 63104, 70469, 200157, 228017, 40377, 154976, 265955, 142298, 126337, 187763, 116221, 96536, 267536, 47194, 102148, 86056, 177531, 124176, 113843, 28292, 265955, 256479, 154201, 19219, 226506, 154201, 1, 256479, 200157, 262080, 2543, 193784, 228014, 265955, 256479, 69677, 266744, 199415, 257011, 200157, 171106, 114599, 113843, 159115, 170361, 154201, 256234, 200157, 130806, 200157, 154201, 228017, 40377, 154201, 228017, 40377, 113843, 271789, 58215, 265955, 119704, 154201, 33975, 200157, 205781, 200157, 224379, 256479, 132276, 113843, 92294, 170361, 154201, 135818, 200157, 11260, 228017, 40377, 226506, 51050, 200157, 256479, 273229, 70498, 170361, 193199, 16992, 124473, 200157, 154201, 205075]","[27479, 52758, 9593, 42648, 44646, 13543, 50801, 21666, 60518, 64365, 46532, 32648, 11870, 7976, 677, 22654, 59208, 19106, 39798, 6022]","[0.9208367466926575, 0.5991209745407104, 0.5697671175003052, 0.5058843493461609, 0.4332846701145172, 0.43209946155548096, 0.4083470404148102, 0.3777143359184265, 0.36745890974998474, 0.36508697271347046, 0.343335896730423, 0.334538072347641, 0.33188268542289734, 0.32894569635391235, 0.3152727782726288, 0.2528184950351715, 0.25032153725624084, 0.24932122230529785, 0.24719524383544922, 0.24712055921554565]",11,BookChapter,A Graph Theoretic Approach to Atomic Displacements in Fullerenes,a graph theoretic approach to atomic displacements in fullerenes,None,96.0,"The recently developed idea of analyzing complex networks in terms of node displacement due to vibration (Estrada and Hatano, Chem Phys Lett 486:166–170, 2010a) is applied to fullerenes. The fact that the ramafullerenes (fullerenes of Ramanujan graphs) are limited to fullerenes with relatively small number of C atoms is explained from the point of view of the node displacement. The node displacement is also shown to indicate the stability of isomers of C40 fullerenes. It is suggested from the analysis of local node displacement that instability of fullerenes mainly comes from pentagon-rich areas of the molecules.",the recently developed idea of analyzing complex networks in terms of node displacement due to vibration estrada and hatano chem phys lett 486 166 170 2010a is applied to fullerenes the fact that the ramafullerenes fullerenes of ramanujan graphs are limited to fullerenes with relatively small number of c atoms is explained from the point of view of the node displacement the node displacement is also shown to indicate the stability of isomers of c40 fullerenes it is suggested from the analysis of local node displacement that instability of fullerenes mainly comes from pentagon rich areas of the molecules,"[complex network, laplacian matrix, node, mathematics, displacement, regular graph, ramanujan s sum, statistical physics, fullerene, classical mechanics, resistance distance]","{""IndexLength"":96,""InvertedIndex"":{""The"":[0,28,59],""recently"":[1],""developed"":[2],""idea"":[3],""of"":[4,10,34,45,53,55,69,71,80,86,93],""analyzing"":[5],""complex"":[6],""networks"":[7],""in"":[8],""terms"":[9],""node"":[11,57,60,82],""displacement"":[12,61,83],""due"":[13],""to"":[14,26,39,65],""vibration"":[15],""(Estrada"":[16],""and"":[17],""Hatano,"":[18],""Chem"":[19],""Phys"":[20],""Lett"":[21],""486:166–170,"":[22],""2010a)"":[23],""is"":[24,48,62,75],""applied"":[25],""fullerenes."":[27,73],""fact"":[29],""that"":[30,84],""the"":[31,51,56,67,78,94],""ramafullerenes"":[32],""(fullerenes"":[33],""Ramanujan"":[35],""graphs)"":[36],""are"":[37],""limited"":[38],""fullerenes"":[4

In [23]:
json_data = data[['original_title','doc_type','journal_name','indexed_abstract']].copy()
json_data.columns = ['title','doc_type','journal','abstract']
json_data.to_json('test_json_V2.json', orient='records')

#### Reading in like an input

In [94]:
model_path = "./model_to_api/container/model_files/"

In [97]:
# Load the dictionaries
with open(os.path.join(model_path, "topics_vocab.pkl"), "rb") as f:
    target_vocab = pickle.load(f)
    
target_vocab_inv = {j:i for i,j in target_vocab.items()}

print("Loaded target vocab")

with open(os.path.join(model_path, "doc_type_vocab.pkl"), "rb") as f:
    doc_vocab = pickle.load(f)
    
doc_vocab_inv = {j:i for i,j in doc_vocab.items()}

print("Loaded doc_type vocab")

with open(os.path.join(model_path, "journal_name_vocab.pkl"), "rb") as f:
    journal_vocab = pickle.load(f)
    
journal_vocab_inv = {j:i for i,j in journal_vocab.items()}

print("Loaded journal vocab")

with open(os.path.join(model_path, "paper_title_vocab.pkl"), "rb") as f:
    title_vocab = pickle.load(f)
    
title_vocab_inv = {j:i for i,j in title_vocab.items()}

print("Loaded title vocab")

with open(os.path.join(model_path, "tag_id_vocab.pkl"), "rb") as f:
    tag_id_vocab = pickle.load(f)

Loaded target vocab
Loaded doc_type vocab
Loaded journal vocab
Loaded title vocab


In [93]:
def invert_abstract_to_abstract(invert_abstract):
    invert_abstract = json.loads(invert_abstract)
    ab_len = invert_abstract['IndexLength']
    
    if 30 < ab_len < 1000:
        abstract = [" "]*ab_len
        for key, value in invert_abstract['InvertedIndex'].items():
            for i in value:
                abstract[i] = key
        final_abstract = " ".join(abstract)
    else:
        final_abstract = None
    return final_abstract

def clean_abstract(abstract, inverted=True):
    if inverted:
        if abstract:
            abstract = invert_abstract_to_abstract(abstract)
        else:
            pass
    else:
        pass
    abstract = clean_text(abstract)
    return abstract

def clean_text(text):
    try:
        text = text.lower()

        text = re.sub('[^a-zA-Z0-9 ]+', ' ', text)
        text = re.sub(' +', ' ', text)
        text = text.strip()
        
    except:
        text = ""
    return text

def try_lowercase(text):
    try:
        text = text.lower()
    except:
        pass
    return text

def tokenize_feature(feature, feature_name='doc_type'):
    if feature_name=='doc_type':
        vocab = doc_vocab
    else:
        vocab = journal_vocab
    unk_token_id = vocab.get('[UNK]')
    none_token_id = vocab.get('[NONE]')
    if feature:
        token_feature = [vocab.get(feature, unk_token_id)]
    else:
        token_feature = [none_token_id]
    return token_feature

def tokenize_title(feature):
    split_feature = feature.split(" ")
    vocab = title_vocab
    unk_token_id = vocab.get('[UNK]')
    none_token_id = vocab.get('[NONE]')
    if feature:
        token_feature = [vocab.get(x, unk_token_id) for x in split_feature]
    else:
        token_feature = [none_token_id]
    return token_feature

In [103]:
input_df = pd.read_json('test_json_V2.json', orient='records').reset_index(drop=True)

In [106]:
input_df

,title,doc_type,journal,abstract,inverted_abstract,paper_title_tok,abstract_tok,doc_type_tok,journal_tok
0,a graph theoretic approach to atomic displacements in fullerenes,BookChapter,None,the recently developed idea of analyzing complex networks in terms of node displacement due to vibration estrada and hatano chem phys lett 486 166 170 2010a is applied to fullerenes the fact that the ramafullerenes fullerenes of ramanujan graphs are limited to fullerenes with relatively small number of c atoms is explained from the point of view of the node displacement the node displacement is also shown to indicate the stability of isomers of c40 fullerenes it is suggested from the analysis of local node displacement that instability of fullerenes mainly comes from pentagon rich areas of the molecules,True,"[239113, 92190, 154991, 132270, 265955, 114561, 51847, 63104, 256479]","[154201, 47075, 139417, 222311, 200157, 90902, 11995, 148679, 63104, 70469, 200157, 228017, 40377, 154976, 265955, 142298, 126337, 187763, 116221, 96536, 267536, 47194, 102148, 86056, 177531, 124176, 113843, 28292, 265955, 256479, 154201, 19219, 226506, 154201, 1, 256479, 200157, 262080, 2543, 193784, 228014, 265955, 256479, 69677, 266744, 199415, 257011, 200157, 171106, 114599, 113843, 159115, 170361, 154201, 256234, 200157, 130806, 200157, 154201, 228017, 40377, 154201, 228017, 40377, 113843, 271789, 58215, 265955, 119704, 154201, 33975, 200157, 205781, 200157, 224379, 256479, 132276, 113843, 92294, 170361, 154201, 135818, 200157, 11260, 228017, 40377, 226506, 51050, 200157, 256479, 273229, 70498, 170361, 193199, 16992, 124473, 200157, 154201, 205075]",[8],[2]
1,sectional anatomy of the human temporal bone,Journal,folia morphologica,,True,"[58192, 126692, 200157, 154201, 69678, 255482, 2487]",[2],[3],[3165]
2,pakistan response towards terrorism a case study of musharraf regime,Thesis,None,the ranging course of terrorism banishing peace and security prospects of today s pakistan is seen as a domestic effluent of its own flawed policies bad governance and lack of social justice and rule of law in society and widening gulf of trust between the rulers and the ruled the study focused on policies and performance of the musharraf government since assuming the mantle of front ranking ally of the united states in its so called war on terror the causes of reversal of pre nine eleven position on afghanistan and support of its taliban s rulers are examined in the light of the geo strategic compulsions of that crucial time and the structural weakness of military rule that needed external props for legitimacy the flaws of the response to the terrorist challenges are traced to its total dependence on the hard option to the total neglect of the human factor from which the thesis develops its argument for a holistic approach to security in which the people occupy a central position thesis approach is also shown to hold the solutions for eliminating the causes of extremism on which terrorism feeds and grows in sum the study deconstructs musharraf s regime s response to terrorism by examining the conceptual mould of the strategic players in the country and postulates a holistic and integrated security framework to deal with terrorism on a pro active and sustainable basis an approach such as this would logically entail the redefining of the role of the state vis a vis its people as the fulcrum and medium of ensuring traditional and non traditional security of the country,True,"[247096, 63112, 86555, 62423, 239113, 199399, 68895, 200157, 280974, 56710]","[154201, 153540, 85053, 200157, 62423, 143063, 154250, 187763, 46948, 86572, 200157, 86558, 278048, 247096, 113843, 23366, 244521, 239113, 221564, 176060, 200157, 135821, 39638, 12022, 87345, 120276, 17672, 187763, 12048, 200157, 142282, 277329, 187763, 200167, 200157, 273948, 63104, 261129, 187763, 279579, 155008, 200157, 120299, 113846, 154201, 152911, 187763, 154201, 226673, 154201, 68895, 90654, 114560, 87345, 187763, 223011, 2001

In [105]:
input_df['title'] = input_df['title'].apply(clean_text)
input_df['abstract'] = input_df.apply(lambda x: clean_abstract(x.abstract, x.inverted_abstract), axis=1)
input_df['journal'] = input_df['journal'].apply(try_lowercase)
input_df['paper_title_tok'] = input_df['title'].apply(tokenize_title)
input_df['abstract_tok'] = input_df['abstract'].apply(tokenize_title)
input_df['doc_type_tok'] = input_df['doc_type'].apply(tokenize_feature, args=('doc_type',))
input_df['journal_tok'] = input_df['journal'].apply(tokenize_feature, args=('journal',))

#### Testing API

In [1]:
import requests
import json
import multiprocessing as mp
import mag_functions as F

In [2]:
with open("test_json_V2.json", "rb") as f:
    input_json = json.load(f)

In [3]:
%%time
F.get_tags(input_json, 1)[1]

CPU times: user 31.7 ms, sys: 12.3 ms, total: 44 ms
Wall time: 694 ms


[{'tags': ['fullerene',
   'node',
   'displacement',
   'ramanujan s sum',
   'pentagon',
   'computational chemistry',
   'physics',
   'combinatorics',
   'mathematics'],
  'scores': [0.920840859413147,
   0.5919812321662903,
   0.5425196290016174,
   0.5040149092674255,
   0.46491581201553345,
   0.40673425793647766,
   0.39379405975341797,
   0.3785310983657837,
   0.35838180780410767],
  'tag_ids': [162862793,
   62611344,
   107551265,
   131220774,
   2779965921,
   147597530,
   121332964,
   114614502,
   33923547]},
 {'tags': ['anatomy', 'temporal bone', 'human anatomy', 'medicine', 'biology'],
  'scores': [0.6792266964912415,
   0.6614392995834351,
   0.45784658193588257,
   0.3365514278411865,
   0.3269321322441101],
  'tag_ids': [105702510, 2777939775, 2985650651, 71924100, 86803240]},
 {'tags': ['terrorism',
   'legitimacy',
   'political science',
   'domestic terrorism',
   'political economy',
   'argument',
   'position',
   'law'],
  'scores': [0.7742798328399658,
 